## Import and Load data file

In [40]:
# -*- coding: utf-8 -*-
import nltk
from tensorflow.python.keras.layers import LayerNormalization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
from tensorflow.keras.models import Sequential
import numpy as np
#from keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import codecs


words=[] #liste des mots des patterns
classes = [] # va contenir les tags
documents = [] #va contenir liste des sentences d'un pattern suivi de son tag
ignore_words = ['?', '!','-']
#data_file = codecs.open('intents.json',encoding='utf-8').read()
#data_file = open('intents.json').read()
data_file = codecs.open('intents.json',encoding='utf-8').read()
intents = json.loads(data_file)

## Pre-process data

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [41]:


from nltk.tokenize import word_tokenize
from nltk import RegexpTokenizer
toknizer = RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')

#ignore_words = ['?', '!','.',',','-','_','\'']
#data_file = open('intents.json').read()
data_file = codecs.open('intents.json',encoding='utf-8').read()
intents = json.loads(data_file)
#intents is a dictionnary sous forme {"":""} intents is key and we have one array as value
for intent in intents['intents']: # parcours d'un dictionnaire: name_dic['key']          #print(intent)
    for pattern in intent['patterns']:      #print(pattern)
        
   
        #tokenize each word
        #w=toknizer.tokenize(pattern)
        w = nltk.word_tokenize(pattern,language ='french') #  w est une liste des mots
        print(w)
        words.extend(w) # ajouter chaque w à la liste words ;nous avons utilisé extend car w est une liste: fusionner deux listes
        #add documents in the corpus
        documents.append((w, intent['tag'])) #ajouter une tuple (w et sa correspondence tag)dans docueents à la fin

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

['Bonjour']
['Bonne', 'journée']
['Allô']
['Bonsoir']
['Salut']
['ça', 'roule', '?']
['Quel', 'âge', 'as-tu', '?']
["C'est", 'quand', 'ton', 'anniversaire', '?']
['Quand', 'es-tu', 'né', '?']
['Que', 'fais-tu', 'ce', 'week-end', '?']
['Tu', 'veux', "qu'on", 'fasse', 'un', 'truc', 'ensemble', '?']
['Quels', 'sont', 'tes', 'plans', 'pour', 'cette', 'semaine']
['Quel', 'est', 'ton', 'prénom', '?']
['Comment', 'tu', "t'appelles", '?']
['Qui', 'es-tu', '?']
['bye']
['Bonne', 'soirée']
['À', 'tout', 'à', 'l', '’', 'heure']
['Bonne', 'nuit']
['au', 'revoir']
['à', 'bientot']
['Comment', 'pourriez-vous', "m'aider", '?']
['Que', 'pouvez-vous', 'faire', '?']
['Quelle', 'aide', 'apportez-vous', '?']
['Comment', 'pouvez-vous', 'être', 'utile', '?']
['Quel', 'soutien', 'est', 'offert']
['Comment', 'vérifier', 'les', 'effets', 'indésirables', 'des', 'médicaments', '?']
['Ouvrir', 'le', 'module', 'sur', 'les', 'médicaments', 'indésirables']
['Donnez-moi', 'une', 'liste', 'des', 'médicaments', 'provoq

lemmatize each word and remove duplicate words from the list

In [42]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

51 documents
12 classes ['adverse_drug', 'age', 'appointment', 'blood_pressure', 'date', 'goodbye', 'greeting', 'hospital_search', 'name', 'options', 'payement', 'pharmacy_search']
124 unique lemmatized words ['accépter', 'adaptés', 'aide', 'aidez-moi', 'allô', 'anniversaire', 'apportez-vous', 'artérielle', 'as-tu', 'au', 'bientot', 'bonjour', 'bonne', 'bonsoir', 'bye', "c'est", 'carte', 'ce', 'cette', 'clinique', 'comment', 'comportement', "d'effet", "d'hôpital", "d'un", 'de', 'donnez-moi', 'données', 'détails', 'effet', 'effets', 'enregistrer', 'ensemble', 'es-tu', 'est', 'factures', 'faire', 'fais-tu', 'fasse', 'gestion', 'heure', 'hôpital', 'indésirable', 'indésirables', 'je', 'journée', 'l', 'la', 'le', 'liste', 'listez', 'liée', 'localiser', "m'aider", 'module', 'médicaments', "n'ont", 'nuit', 'né', 'offert', 'ouvrir', 'pa', 'patient', 'payement', 'payer', 'peux', 'pharmacie', 'pharmacy', 'plan', 'pour', 'pourriez-vous', 'pouvez-vous', 'prendre', 'pression', 'provoquant', 'proxim

## Create training and testing data

In [43]:

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    #print(output_row)
    
    training.append([bag, output_row])
    #print(training)
# shuffle our features and turn into np.array
random.shuffle(training)  #Shuffle a list (reorganize the order of the list items): aléatoire,la liste va changer ne trvaille pas sur une copié
training = np.array(training)
#print(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
#print(train_x)
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-43-ba011560979a>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


## Create Model

In [44]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
11/11 [==============================] - 1s 6ms/step - loss: 2.5418 - accuracy: 0.0588
Epoch 2/200
11/11 [==============================] - 0s 7ms/step - loss: 2.5232 - accuracy: 0.1373
Epoch 3/200
11/11 [==============================] - 0s 7ms/step - loss: 2.3377 - accuracy: 0.1961
Epoch 4/200
11/11 [==============================] - 0s 8ms/step - loss: 2.2423 - accuracy: 0.2941
Epoch 5/200
11/11 [==============================] - 0s 6ms/step - loss: 2.2295 - accuracy: 0.3333
Epoch 6/200
11/11 [==============================] - 0s 6ms/step - loss: 2.1177 - accuracy: 0.3725
Epoch 7/200
11/11 [==============================] - 0s 8ms/step - loss: 1.9800 - accuracy: 0.4706: 0s - loss: 1.9263 - accuracy: 0.55
Epoch 8/200
11/11 [==============================] - 0s 7ms/step - loss: 1.8356 - accuracy: 0.4706
Epoch 9/200
11/11 [==============================] - 0s 7ms/step - loss: 1.7814 - accuracy: 0.4706: 0s - loss: 1.7665 - accuracy: 0.48
Epoch 10/200
11/11 [=================

11/11 [==============================] - 0s 5ms/step - loss: 0.1005 - accuracy: 0.9804
Epoch 82/200
11/11 [==============================] - 0s 6ms/step - loss: 0.1241 - accuracy: 0.9804
Epoch 83/200
11/11 [==============================] - 0s 7ms/step - loss: 0.0174 - accuracy: 1.0000
Epoch 84/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0713 - accuracy: 0.9804
Epoch 85/200
11/11 [==============================] - 0s 5ms/step - loss: 0.1025 - accuracy: 0.9608
Epoch 86/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0338 - accuracy: 1.0000
Epoch 87/200
11/11 [==============================] - 0s 7ms/step - loss: 0.0610 - accuracy: 0.9804
Epoch 88/200
11/11 [==============================] - 0s 5ms/step - loss: 0.1521 - accuracy: 0.9804
Epoch 89/200
11/11 [==============================] - 0s 5ms/step - loss: 0.1017 - accuracy: 0.9804
Epoch 90/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0317 - accuracy: 1.0000
Epoch 91/200


11/11 [==============================] - 0s 5ms/step - loss: 0.0199 - accuracy: 1.0000
Epoch 163/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0516 - accuracy: 1.0000
Epoch 164/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0323 - accuracy: 1.0000
Epoch 165/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0461 - accuracy: 0.9804
Epoch 166/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0318 - accuracy: 1.0000
Epoch 167/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0151 - accuracy: 1.0000
Epoch 168/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0490 - accuracy: 1.0000
Epoch 169/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0598 - accuracy: 0.9804
Epoch 170/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0178 - accuracy: 1.0000
Epoch 171/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0152 - accuracy: 1.0000
Epoc

In [45]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(codecs.open('intents.json',encoding='utf-8').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [46]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [47]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [48]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [49]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [50]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

## GUI Interface

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Chatty: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

root = Tk()
root.title("Chatty")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)


#Create Chat window
ChatLog = Text(root, bd=0, bg="white", height="8", width="70", font="Arial",)

ChatLog.config(state=DISABLED)
ChatLog.config(background='#d3f0f0')

#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#c27ef4",activebackground="#fcdbf4",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial", background="#e0cefb")
#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=300)
ChatLog.place(x=8,y=9, height=386, width=370)
EntryBox.place(x=130, y=401, height=40, width=265)
SendButton.place(x=6, y=401, height=40)
root.mainloop()
